In [302]:
import numpy as np
import pandas as pd
import re
import os

In [303]:
sess1 = pd.read_csv(r"C:\Users\Masha\Documents\Undesranding\AgingSpeed_randomization_001.csv", sep=";", encoding = 'utf-8')
sess2 = pd.read_csv(r"C:\Users\Masha\Documents\Undesranding\AgingSpeed_randomization_002.csv", sep=";", encoding = 'utf-8')
sess3 = pd.read_csv(r"C:\Users\Masha\Documents\Undesranding\AgingSpeed_randomization_003.csv", sep=";", encoding = 'utf-8')

In [304]:
sess1['Question'][87]

'Где супруги решили провести отпуск?'

In [305]:
sess2.head()
sess3.shape

(100, 8)

In [306]:
sess3.head()
sess3.shape

(100, 8)

In [307]:
s1 = open(r'C:\Users\Masha\Documents\Undesranding\Sent_type\Session1_rand1.txt', 'r', encoding = 'windows-1251')
s2 = open(r'C:\Users\Masha\Documents\Undesranding\Sent_type\Session2_rand1.txt', 'r', encoding = 'windows-1251')
s3 = open(r'C:\Users\Masha\Documents\Undesranding\Sent_type\Session3_rand5.txt', 'r', encoding = 'windows-1251')

sent1 = s1.read().split(';')
sent2 = s2.read().split(';')
sent3 = s3.read().split(';')

s1.close()
s2.close()
s3.close()

In [308]:
types = {}

In [309]:
def get_type(sess, sent, types):
    
    for line in sent:
        if '?' in line:
            res = re.search('> \"(.+?)\", <', line)
            quest = res.group(1)
            
            for el in sess['Question']:
            
                if quest == el:
                    num = line[2:8]
                    res = re.search('[1-90]+ +([A-Za-z]+)', str(sess[sess['Question'] == el]['Type']))
                    t = res.group(1)
                    types[num] = t
    return types 

In [341]:
get_type(sess1, sent1, types)
get_type(sess3, sent3, types)
get_type(sess2, sent2, types)
types['380910'] = 'Fillers'

In [311]:
def get_errors(f):
    er = {}
    for line in f:
        if '-' in line:
            
            if types[line[:6]] in er:
                er[types[line[:6]]] += 1
            else:
                er[types[line[:6]]] = 1
            #print(line[:6])
    return er

In [312]:
path = 'C:\\Users\\Masha\\Documents\\Undesranding\\Results\\'

all = {}

for roots, dirs, files in os.walk(path):
    for d in dirs:
        
        for file in os.listdir(roots + '\\' + d):
            if '.azk' in file:
                a = open(roots + '\\' + d + '\\' + file, 'r')
                f = a.readlines()
                #print(roots + '\\' + d + '\\' + file)                
                if 'FRR' in file:
                    if '!' in f[7]:
                        f = f[15:]
                    else:
                        f = f[5:]

                elif '!' in f[7]:
                    f = f[61:]
                else:
                    f = f[51:]
                    
                if 'M' in roots:
                    all[roots[-3:]+' '+d] = get_errors(f)

                elif 'S' in roots:
                    all[roots[-3:]+' '+d] = get_errors(f)                    
                a.close()

In [340]:
head = ['Name', 'GenNP', 'OVS', 'SVO', 'ObjRel', 'Refl', 'SubjRel', 'Fillers', 'Non filler', 'All']
table = []
for name in all:
    time = [name]
    
    for i in head[1:8]:
        if i in all[name]:
            time.append(all[name][i])
        else:
            time.append(0)
    time.append(sum(time[1:7]))
    time.append(sum(time[1:8]))
    table.append(time)

In [314]:
itog = pd.DataFrame(sorted(table), columns=head)

In [315]:
itog['GenNP'] = round(100 - itog['GenNP']/30*100, 2)
itog['OVS'] = round(100 - itog['OVS']/10*100, 2)
itog['SVO'] = round(100 - itog['SVO']/6*100, 2)
itog['ObjRel'] = round(100 - itog['ObjRel']/9*100, 2)
itog['SubjRel'] = round(100 - itog['SubjRel']/9*100, 2)
itog['Refl'] = round(100 - itog['Refl']/16*100, 2)
itog['Fillers'] = round(100 - itog['Fillers']/26*100, 2)
itog['Non filler'] = round(100 - itog['Non filler']/74*100, 2)
itog['All'] = round(100 - itog['All'], 2)

In [317]:
names = sorted(list(set([i[:3] for i in itog['Name']])))
itog.shape[0]

117

In [331]:
aver_fast = pd.DataFrame([], index = names, columns=head[1:])

for p in head[1:]:
    i = 0
    for n in names:
        aver_fast[p][n] = round(float(itog[p][i]) - float(itog[p][i + 1]), 2)
        i += 3

In [319]:
spr = itog[2::3]
aver = itog[::3]
fast = itog[1::3]

In [320]:
stat_spr = pd.DataFrame([], index = ['Mean', 'Med', 'SD', 'Min', 'Max'], columns=head[1:])

for p in head[1:]:
    stat_spr[p]['Mean'] = round(spr[p].mean(), 2)
    stat_spr[p]['Med'] = round(spr[p].median(), 2)
    stat_spr[p]['SD'] = round(spr[p].std(), 2)
    stat_spr[p]['Min'] = round(spr[p].min(), 2)
    stat_spr[p]['Max'] = round(spr[p].max(), 2)

In [330]:
stat_aver = pd.DataFrame([], index = ['Mean', 'Med', 'SD', 'Min', 'Max'], columns=head[1:])

for p in head[1:]:
    stat_aver[p]['Mean'] = round(aver[p].mean(), 2)
    stat_aver[p]['Med'] = round(aver[p].median(), 2)
    stat_aver[p]['SD'] = round(aver[p].std(), 2)
    stat_aver[p]['Min'] = round(aver[p].min(), 2)
    stat_aver[p]['Max'] = round(aver[p].max(), 2)

In [327]:
stat_fast = pd.DataFrame([], index = ['Mean', 'Med', 'SD', 'Min', 'Max'], columns=head[1:])

for p in head[1:]:
    stat_fast[p]['Mean'] = round(fast[p].mean(), 2)
    stat_fast[p]['Med'] = round(fast[p].median(), 2)
    stat_fast[p]['SD'] = round(fast[p].std(), 2)
    stat_fast[p]['Min'] = round(fast[p].min(), 2)
    stat_fast[p]['Max'] = round(fast[p].max(), 2)

In [325]:
stat_aver_fast = pd.DataFrame([], index = ['Mean', 'Med', 'SD', 'Min', 'Max'], columns=head[1:])
for p in head[1:]:
    stat_aver_fast[p]['Mean'] = round(aver_fast[p].mean(), 2)
    stat_aver_fast[p]['Med'] = round(aver_fast[p].median(), 2)
    stat_aver_fast[p]['SD'] = round(aver_fast[p].std(), 2)
    stat_aver_fast[p]['Min'] = round(aver_fast[p].min(), 2)
    stat_aver_fast[p]['Max'] = round(aver_fast[p].max(), 2)

# Таблицы с данными

### Таблица % правильных ответов по всем типам предложений

In [343]:
itog[:60]

,Name,GenNP,OVS,SVO,ObjRel,Refl,SubjRel,Fillers,Non filler,All
0,M01 Average,70.00,100.0,100.00,100.00,87.50,55.56,88.46,79.73,82
1,M01 Fast,53.33,100.0,100.00,77.78,81.25,66.67,100.00,70.27,78
2,M01 SPR,66.67,100.0,100.00,77.78,100.00,88.89,100.00,82.43,87
3,M02 Average,73.33,90.0,100.00,88.89,87.50,88.89,100.00,82.43,87
4,M02 Fast,60.00,100.0,100.00,100.00,87.50,88.89,92.31,79.73,83
5,M02 SPR,73.33,90.0,83.33,66.67,100.00,88.89,96.15,81.08,85
6,M03 Average,96.67,100.0,100.00,77.78,93.75,88.89,100.00,93.24,95
7,M03 Fast,73.33,90.0,100.00,88.89,100.00,77.78,88.46,83.78,85
8,M03 SPR,96.67,100.0,100.00,77.78,100.00,100.00,100.00,95.95,97
9,M04 Average,90.00,90.0,100.00,88.89,87.50,100.00,96.15,90.54,92


In [344]:
itog[60:]

,Name,GenNP,OVS,SVO,ObjRel,Refl,SubjRel,Fillers,Non filler,All
60,S05 Average,63.33,70.0,83.33,55.56,75.00,77.78,80.77,66.22,70
61,S05 Fast,46.67,60.0,83.33,44.44,68.75,44.44,88.46,51.35,61
62,S05 SPR,60.00,80.0,66.67,77.78,87.50,66.67,100.00,68.92,77
63,S06 Average,70.00,90.0,100.00,77.78,100.00,88.89,88.46,82.43,84
64,S06 Fast,70.00,70.0,100.00,66.67,100.00,88.89,100.00,78.38,84
65,S06 SPR,76.67,90.0,100.00,55.56,93.75,100.00,100.00,82.43,87
66,S07 Average,70.00,90.0,100.00,66.67,93.75,100.00,96.15,81.08,85
67,S07 Fast,60.00,100.0,83.33,88.89,81.25,66.67,96.15,72.97,79
68,S07 SPR,53.33,90.0,100.00,77.78,100.00,100.00,84.62,77.03,79
69,S08 Average,66.67,40.0,83.33,77.78,81.25,77.78,65.38,67.57,67


### Разница Average - Fast по всем типам для каждого человека

In [334]:
aver_fast

,GenNP,OVS,SVO,ObjRel,Refl,SubjRel,Fillers,Non filler,All
M01,16.67,0,0,22.22,6.25,-11.11,-11.54,9.46,4
M02,13.33,-10,0,-11.11,0,0,7.69,2.7,4
M03,23.34,10,0,-11.11,-6.25,11.11,11.54,9.46,10
M04,10,0,0,0,6.25,33.33,3.84,9.46,8
M05,10,-20,0,33.33,6.25,44.45,3.85,12.17,10
M06,36.67,-20,-16.67,11.11,0,22.22,0,14.87,11
M07,3.34,10,-16.67,22.22,0,-22.22,15.38,1.35,5
M08,-16.67,0,0,0,-12.5,0,0,-9.46,-7
M09,3.33,10,-33.33,0,-6.25,-22.22,3.84,-4.05,-2
M10,13.34,10,0,11.11,0,-11.11,3.85,6.76,6


### Статистика SPR

In [336]:
stat_spr

,GenNP,OVS,SVO,ObjRel,Refl,SubjRel,Fillers,Non filler,All
Mean,67.52,87.18,92.73,70.09,87.5,81.77,93.59,75.95,80.54
Med,66.67,90,100,66.67,87.5,88.89,92.31,75.68,80
SD,11.41,14.32,11.34,17.13,11.47,14.3,5.6,8.91,7.49
Min,46.67,50,66.67,33.33,56.25,33.33,80.77,55.41,63
Max,96.67,100,100,100,100,100,100,95.95,97


### Статистика Average

In [337]:
stat_aver

,GenNP,OVS,SVO,ObjRel,Refl,SubjRel,Fillers,Non filler,All
Mean,67.86,86.41,92.31,72.94,85.74,81.48,90.73,75.88,79.74
Med,66.67,90,100,77.78,87.5,88.89,92.31,75.68,80
SD,12.39,15.64,11.37,18.7,9.29,15.36,9.45,10.16,9.29
Min,43.33,40,66.67,33.33,62.5,44.44,53.85,48.65,50
Max,100,100,100,100,100,100,100,97.3,98


### Статистика Fast

In [338]:
stat_fast

,GenNP,OVS,SVO,ObjRel,Refl,SubjRel,Fillers,Non filler,All
Mean,64.19,82.05,93.59,72.08,82.53,76.64,91.42,72.52,77.44
Med,66.67,90,100,77.78,81.25,77.78,92.31,72.97,78
SD,11.02,17.65,13.03,19.22,12.6,15.25,8.93,10,8.9
Min,43.33,40,33.33,22.22,50,44.44,46.15,50,49
Max,96.67,100,100,100,100,100,100,97.3,97


### Статистика Average - Fast

In [339]:
stat_aver_fast

,GenNP,OVS,SVO,ObjRel,Refl,SubjRel,Fillers,Non filler,All
Mean,3.67,4.36,-1.28,0.85,3.21,4.84,-0.69,3.36,2.31
Med,3.33,0,0,0,0,0,0,2.7,2
SD,11.39,14.65,15,19.81,10.81,19.21,7.53,7.25,5.98
Min,-20,-20,-33.33,-44.45,-18.75,-33.34,-15.39,-10.81,-10
Max,36.67,50,33.34,55.56,25,44.45,15.38,14.87,12
